In [9]:
import pandas as pd
import os

# ==============================
# FILE PATHS
# ==============================
BASE_PATH = r"C:\Users\manu.chopra\OneDrive - Reliance Corporate IT Park Limited\Documents\ITSM_AI_Project"

SITES_FILE = os.path.join(BASE_PATH, "Sites.xlsx")
INVENTORY_FILE = os.path.join(BASE_PATH, "Inventory.xlsx")
COMPLAINTS_FILE = os.path.join(BASE_PATH, "Complaints.xlsx")

# ==============================
# LOAD MASTER DATA
# ==============================
sites_df = pd.read_excel(SITES_FILE)
inventory_df = pd.read_excel(INVENTORY_FILE)

# ------------------------------
# CLEAN DATA
# ------------------------------
sites_df.columns = sites_df.columns.str.strip()
inventory_df.columns = inventory_df.columns.str.strip()

sites_df["Site"] = sites_df["Site"].astype(str).str.strip().str.upper()
inventory_df["Site"] = inventory_df["Site"].astype(str).str.strip().str.upper()
inventory_df["Tag"] = inventory_df["Tag"].astype(str).str.strip().str.upper()
inventory_df["Device"] = inventory_df["Device"].astype(str).str.strip()

# ==============================
# LOAD OR CREATE COMPLAINTS FILE
# ==============================
if os.path.exists(COMPLAINTS_FILE):
    complaints_df = pd.read_excel(COMPLAINTS_FILE)
else:
    complaints_df = pd.DataFrame(columns=[
        "Complaint", "Site", "Manager", "STORESTATE",
        "STORECITY", "STORENAME", "Tag", "Device", "Title"
    ])

# ==============================
# GENERATE COMPLAINT NUMBER
# ==============================
def generate_complaint_no(df):
    if df.empty:
        return "IT00001"
    last = df["Complaint"].iloc[-1]
    num = int(last.replace("IT", ""))
    return f"IT{num+1:05d}"

# ==============================
# CHATBOT START
# ==============================
print("\n🤖 Welcome to Virtual IT Support Chatbot")
print("Enter Site Code OR Asset Tag (type 'exit' to quit)\n")

while True:
    user_input = input("Enter Site or Tag: ").strip().upper()

    if user_input == "EXIT":
        print("\nThank you. Session closed.")
        break

    # ------------------------------
    # SITE ENTERED
    # ------------------------------
    site_row = sites_df[sites_df["Site"] == user_input]

    if not site_row.empty:
        site = user_input
        print("✅ Site detected:", site_row["STORENAME"].values[0])

        tag = input("Enter Tag ID: ").strip().upper()
        inv_row = inventory_df[inventory_df["Tag"] == tag]

        if inv_row.empty:
            print("❌ Tag not found. Please restart and try again.\n")
            continue

        # 🔒 VALIDATE TAG BELONGS TO SITE
        if inv_row["Site"].values[0] != site:
            print("❌ Tag does NOT belong to this site.")
            print("Please provide correct Tag ID for site:", site)
            print()
            continue

    else:
        # ------------------------------
        # TAG ENTERED
        # ------------------------------
        inv_row = inventory_df[inventory_df["Tag"] == user_input]

        if inv_row.empty:
            print("❌ Site or Tag not found. Please restart.\n")
            continue

        tag = user_input
        site = inv_row["Site"].values[0]
        site_row = sites_df[sites_df["Site"] == site]

        print("✅ Tag detected for site:", site_row["STORENAME"].values[0])

    # ==============================
    # COMMON DETAILS
    # ==============================
    device = inv_row["Device"].values[0]
    title = input("Enter Complaint Title: ").strip()

    complaint_no = generate_complaint_no(complaints_df)

    new_row = {
        "Complaint": complaint_no,
        "Site": site,
        "Manager": site_row["Manager"].values[0],
        "STORESTATE": site_row["STORESTATE"].values[0],
        "STORECITY": site_row["STORECITY"].values[0],
        "STORENAME": site_row["STORENAME"].values[0],
        "Tag": tag,
        "Device": device,
        "Title": title
    }

    complaints_df = pd.concat(
        [complaints_df, pd.DataFrame([new_row])],
        ignore_index=True
    )

    complaints_df.to_excel(COMPLAINTS_FILE, index=False)

    print("\n✅ Complaint Registered Successfully")
    print("📌 Complaint Number:", complaint_no)
    print("Program will now close.")
    print("-" * 60)

    break   # 🛑 EXIT PROGRAM AFTER SUCCESS



🤖 Welcome to Virtual IT Support Chatbot
Enter Site Code OR Asset Tag (type 'exit' to quit)



Enter Site or Tag:  S2463


✅ Site detected: General Stores   ADARI


Enter Tag ID:  IA04042


❌ Tag does NOT belong to this site.
Please provide correct Tag ID for site: S2463



Enter Site or Tag:  IA04927


✅ Tag detected for site: General Stores   ADARI


Enter Complaint Title:  laptop not working



✅ Complaint Registered Successfully
📌 Complaint Number: IT00003
Program will now close.
------------------------------------------------------------


In [11]:
import os
import pandas as pd

BASE_PATH = r"C:\Users\manu.chopra\OneDrive - Reliance Corporate IT Park Limited\Documents\ITSM_AI_Project"

kedb_path = os.path.join(BASE_PATH, "KEDB.xlsx")
solution_path = os.path.join(BASE_PATH, "KEDB_Solution.xlsx")

# Load files
kedb_df = pd.read_excel(kedb_path)
solution_df = pd.read_excel(solution_path)

# Clean columns
kedb_df.columns = kedb_df.columns.str.strip()
solution_df.columns = solution_df.columns.str.strip()

# Merge on KEDB
full_kedb = kedb_df.merge(solution_df, on="KEDB", how="left")

print(full_kedb.head())


     KEDB       FORMAT CATEGORY SUBCATEGORY              ISSUE_TYPE  \
0  KM2985  BACK OFFICE  PRINTER     PRINTER            DRIVER ISSUE   
1  KM2985  BACK OFFICE  PRINTER     PRINTER    FAINT PRINTING ISSUE   
2  KM2985  BACK OFFICE  PRINTER     PRINTER         HARDWARE FAULTY   
3  KM2985  BACK OFFICE  PRINTER     PRINTER     INK REFILLING ISSUE   
4  KM2985  BACK OFFICE  PRINTER     PRINTER  PAPER PICKUP TRY ISSUE   

                                            Solution  
0  1. Turn off the Printer\t\t\n\t\t1. Turn off t...  
1  1. Turn off the Printer\t\t\n\t\t1. Turn off t...  
2  1. Turn off the Printer\t\t\n\t\t1. Turn off t...  
3  1. Turn off the Printer\t\t\n\t\t1. Turn off t...  
4  1. Turn off the Printer\t\t\n\t\t1. Turn off t...  


In [12]:
full_kedb["TEXT"] = (
    full_kedb["FORMAT"].fillna("") + " " +
    full_kedb["CATEGORY"].fillna("") + " " +
    full_kedb["SUBCATEGORY"].fillna("") + " " +
    full_kedb["ISSUE_TYPE"].fillna("")
)

X = full_kedb["TEXT"]
y = full_kedb["ISSUE_TYPE"]


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    lowercase=True,
    stop_words="english",
    ngram_range=(1,2)
)

X_vec = vectorizer.fit_transform(X)


In [14]:
from sklearn.linear_model import LogisticRegression

model_issue = LogisticRegression(max_iter=1000)
model_issue.fit(X_vec, y)


LogisticRegression(max_iter=1000)

In [20]:
import pickle

pickle.dump(vectorizer, open("kedb_vectorizer.pkl", "wb"))
pickle.dump(model_issue, open("kedb_issue_model.pkl", "wb"))


In [24]:
import os
import pandas as pd
import pickle

# ==============================
# BASE PATH (single source of truth)
# ==============================
BASE_PATH = r"C:\Users\manu.chopra\OneDrive - Reliance Corporate IT Park Limited\Documents\ITSM_AI_Project"

def p(file):
    return os.path.join(BASE_PATH, file)

# ==============================
# LOAD MODEL
# ==============================
vectorizer = pickle.load(open(p("kedb_vectorizer.pkl"), "rb"))
model_issue = pickle.load(open(p("kedb_issue_model.pkl"), "rb"))

# ==============================
# LOAD KEDB DATA
# ==============================
kedb_df = pd.read_excel(p("KEDB.xlsx"))
solution_df = pd.read_excel(p("KEDB_Solution.xlsx"))

kedb_df.columns = kedb_df.columns.str.strip()
solution_df.columns = solution_df.columns.str.strip()

full_kedb = kedb_df.merge(solution_df, on="KEDB", how="left")

# ==============================
# CHATBOT START
# ==============================
print("\n🤖 KEDB Virtual Support Engineer")
print("Describe the issue once. System will suggest all relevant solutions.\n")

query = input("Enter problem statement: ").strip()

# ==============================
# PREDICT ISSUE TYPE
# ==============================
vec = vectorizer.transform([query])
predicted_issue = model_issue.predict(vec)[0]

print("\n🔍 Predicted Issue Type:", predicted_issue)

# ==============================
# FILTER KNOWLEDGE BASE
# ==============================
matches = full_kedb[full_kedb["ISSUE_TYPE"] == predicted_issue]

# ==============================
# DISPLAY RESULTS
# ==============================
if matches.empty:
    print("\n❌ No KEDB solutions found.")
else:
    print("\n📘 Recommended Knowledge Articles")
    print("=" * 60)

    for _, row in matches.iterrows():
        print(f"\nKEDB ID     : {row['KEDB']}")
        print(f"Format     : {row['FORMAT']}")
        print(f"Category   : {row['CATEGORY']}")
        print(f"Subcategory: {row['SUBCATEGORY']}")
        print("Solution   :")
        print(f" - {row['Solution']}")

    print("\n✅ Total Solutions Found:", len(matches))

print("\nSession completed. Program closed.")



🤖 KEDB Virtual Support Engineer
Describe the issue once. System will suggest all relevant solutions.



Enter problem statement:  monitor physical damage



🔍 Predicted Issue Type: PHYSICAL DAMAGE

📘 Recommended Knowledge Articles

KEDB ID     : KM2985
Format     : BACK OFFICE
Category   : PRINTER
Subcategory: PRINTER
Solution   :
 - 1. Turn off the Printer		
		1. Turn off the printer and unplug it from the power source.
		2. Disconnect any cables connected to the printer.
2. Physical Check		
		Check for any cracks, dents, or broken parts on the casing, tray, or cover.
		Inspect the paper feed rollers for damage or misalignment.
		Look inside the printer for loose or broken components, including the carriage and ink cartridge slots.
		If the printer was dropped, check for internal dislodged parts.
3.  Remove the Paper Jam		
	1. Access the Paper Tray:	
		Open the paper tray and remove any loose paper.
	2. Clear Paper Path:	
		Carefully remove any jammed paper from the input and output trays. Be gentle to avoid tearing the paper.
	3. Open the Rear Access Door (if available):	
		Open the rear access door or duplexer.
		Gently pull out any ja